In [1]:
from bokeh.charts import (
    Bar,
    show,
    output_file,
    output_notebook,
    )
from bokeh import plotting
import pandas as pd

In [2]:
import simplejson as json

In [3]:
C = "CLINTON"
T = "TRUMP"

In [5]:
out = "output/"
tones = json.load(open(out + "the-first-trump-clinton-presidential-debate-transcript-annotated.json", "r"))
tones_2 = json.load(open(out + "everything-that-was-said-at-the-second-donald-trump-vs-hillary-clinton-debate-highlighted.json","r"))
tones_3 = json.load(open(out + "the-final-trump-clinton-debate-transcript-annotated.json","r"))

In [6]:
def tones_to_df(json):
    """tranforms the response dict to DataFrame"""
    emotional_tones = []

    for k in json:
        if k in ("CLINTON", "TRUMP"):
            want = json[k]["tone_categories"]
            for w in want:
                list_of_d = w["tones"]
                for d in list_of_d:
                    d["candidate"] = k
                    emotional_tones.append(d)
    df = pd.DataFrame(emotional_tones)
    df = df[["candidate", "score", "tone_name"]]
    return df

In [7]:
df_tones = []
for t in (tones, tones_2, tones_3):
    df_tones.append(tones_to_df(t))

In [8]:
def show_tone(df, name, tone):
    """
    helper function to view specific tone
    from a given debate
    """
    return df[ (df["tone_name"] == tone) & (df["candidate"] == name)]

In [72]:
show_tone(df_tones[0], C, "Joy")

,candidate,score,tone_name
3,CLINTON,0.081624,Joy


In [73]:
show_tone(df_tones[0], T, "Joy")

,candidate,score,tone_name
16,TRUMP,0.081609,Joy


In [56]:
output_notebook()

Loading BokehJS ...

In [12]:
f = plotting.figure(width=300, height=300)

In [13]:
def create_grouped_bar_from_df(df, num):
    if num:
        num = int(num)
    else:
        num = "Aggregate"

    b = Bar(df, "tone_name", values='score',
            group="candidate",
            title="Debate %s Tone Analysis" % num,
            plot_height = 600,
            plot_width = 800,
            legend="top_right",
            color = ["#70a6ff", "#ed5757"])
    show(b)

In [14]:
for i, df in enumerate(df_tones):
    create_grouped_bar_from_df(df, i+1)

In [15]:
# All transcripts combined in on df
df_all = pd.concat(df_tones)
df_all_mean = pd.DataFrame(df_all.groupby(["candidate", "tone_name"])["score"].mean()).reset_index()
df_all_median = pd.DataFrame(df_all.groupby(["candidate", "tone_name"])["score"].median()).reset_index()

In [74]:
df_all_mean["aggregate"] = "mean"
df_all_median["aggregate"] = "median"
df_aggregate = pd.concat([df_all_mean, df_all_median])

In [101]:
clinton_agg = df_aggregate[ df_aggregate["candidate"] == C ][["tone_name","score","aggregate"]].set_index(["tone_name","aggregate"])
trump_agg = df_aggregate[ df_aggregate["candidate"] == T ][["tone_name","score","aggregate"]].set_index(["tone_name","aggregate"])
(clinton_agg - trump_agg).sort(["score"], ascending=False)

score
tone_name         aggregate          
Analytical        median     0.493000
                  mean       0.398575
Tentative         mean       0.222690
                  median     0.213000
Fear              median     0.153377
Joy               median     0.077259
Openness          median     0.062000
Emotional Range   mean       0.060990
Openness          mean       0.055712
Fear              mean       0.050522
Anger             median     0.046382
Conscientiousness mean       0.044962
                  median     0.038000
Joy               mean       0.026340
Emotional Range   median     0.010000
Anger             mean       0.002463
Confident         mean       0.000000
                  median     0.000000
Sadness           median    -0.011771
Agreeableness     median    -0.015000
Sadness           mean      -0.020520
Extraversion      mean      -0.024699
                  median    -0.061000
Disgust           mean      -0.063683
Agreeableness     mean      -0.070395
Disgust           median    -0.087023

In [70]:
b = Bar(df_aggregate,
        "tone_name",
        values='score',
        group=["aggregate", "candidate"],
        title="Aggregate Debate Tone Analysis",
        plot_height = 600,
        plot_width = 800,
        legend="top_right",
        ylabel="Score",
        color = ["#70a6ff", "#ed5757", "#025df4", "#f20404"])
show(b)